In [2]:
from models.retiro_model import *
import torch

In [3]:
model = GNOT()

Using Linear Attention
Using Linear Attention
Using Linear Attention


In [4]:
x   = torch.rand([4,100,2]).requires_grad_(True)
x_i = torch.rand([4,1,1])

In [5]:
output = model(x,inputs=x_i)

In [6]:
from train_utils.navier_stokes_autograd import ns_pde_autograd_loss, ns_pde_autograd

In [7]:
pde_losses, derivatives = ns_pde_autograd(x,output,Re=x_i*10)

Let's see if the autograd works for multiple epochs and loss function

In [8]:
from train_utils.dynamic_loss_balancing import RELOBRALO

In [9]:
pde_losses[0].shape

torch.Size([4, 100])

In [10]:
model = GNOT()
epochs = 100
relobralo = RELOBRALO(device='cpu')

optimizer = torch.optim.AdamW(model.parameters(), 
                                    betas=(0.9, 0.999), 
                                    lr=0.01,
                                    weight_decay=0.00005
                                    )
loss_function = torch.nn.MSELoss()
x   = torch.rand([4,100,2])#.requires_grad_(True)
x_i = torch.rand([4,1,1])

for e in range(epochs):
    optimizer.zero_grad()
    x_in = x.clone()
    x_in.requires_grad = True

    output = model(x_in,inputs=x_i)

    supervised_loss = [loss_function(output,torch.ones_like(output))]
    pde_loss_list = ns_pde_autograd_loss(x_in,output,Re=x_i*10)

    all_losses_list = supervised_loss + pde_loss_list


    total_losses_bal = relobralo(loss_list=all_losses_list)
    total_losses_bal.backward()
    optimizer.step()

    print(e, total_losses_bal.item(), all_losses_list[0].item(), all_losses_list[1].item(), all_losses_list[2].item(), all_losses_list[3].item())

    

Using Linear Attention
Using Linear Attention
Using Linear Attention
0 0.5566155910491943 0.31936296820640564 0.09534536302089691 0.09351211786270142 0.047878675162792206
1 4.7147722244262695 2.316746950149536 0.07103215157985687 1.460742473602295 0.8642853498458862
2 10.846285820007324 7.885335922241211 1.3167592287063599 1.2047524452209473 0.4393516778945923
3 12.460443496704102 7.230006694793701 0.15954603254795074 2.7476274967193604 2.3297910690307617
4 7.7360148429870605 7.592007637023926 0.028351102024316788 0.004698147065937519 0.09453578293323517
5 4.3070292472839355 4.262635707855225 0.004743481054902077 0.02526351436972618 0.009613150730729103
6 0.7523016929626465 0.7277841567993164 0.010380929335951805 0.005406122654676437 0.008629374206066132
7 3.601954936981201 3.440579414367676 0.05126090347766876 0.0296825859695673 0.08318401128053665
8 0.5972688794136047 0.5478684902191162 0.010435030795633793 0.0310489721596241 0.00894132163375616
9 1.1194519996643066 0.991003632545471

KeyboardInterrupt: 

In [24]:
sum(all_losses_list)/len(all_losses_list)

tensor(0.0373, grad_fn=<DivBackward0>)

In [45]:
dict3 = dict.fromkeys(['Supervised Loss','PDE 1','PDE 2'],[])

In [41]:
dict3['Supervised Loss'].append(0.5)

In [42]:
dict3

{'Supervised Loss': [0.5, 0.5], 'PDE 1': [0.5, 0.5], 'PDE 2': [0.5, 0.5]}

In [27]:
dict1 = {"losses" : 10, "pde1": 20}
dict2 = {"losses" : 30, "pde1": 420}

In [29]:
dict1dict2

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [59]:
class total_loss_list():
    def __init__(self):
        super(total_loss_list, self).__init__()
        self.dictionary = dict()
        
    def update(self, loss_list):
        for key_name in loss_list.keys():
            if key_name not in self.dictionary.keys():
                self.dictionary[key_name] = []
            self.dictionary[key_name].append(loss_list[key_name])
    
    def fetch_list(self):
        return self.dictionary

In [44]:
total_loss_list = total_loss_list()

In [51]:
total_loss_list.update(dict2)

In [58]:
total_loss_list

__main__.total_loss_list

In [62]:
dict1 = {}
dict1['hello'] = 0
dict1['vd'] = 12

In [63]:
dict1

{'hello': 0, 'vd': 12}

In [65]:
losses = [ 1,2,3]

In [66]:
keys = ['hi', 'bill', 'say']

In [69]:
{keys[i]+'bal':j for i,j in enumerate(losses)}

{'hibal': 1, 'billbal': 2, 'saybal': 3}

In [70]:
import numpy as np

class total_loss_list():
    def __init__(self):
        super(total_loss_list, self).__init__()
        self.dictionary = dict()
        
    def update(self, loss_dict):
        for key_name in loss_dict.keys():
            if key_name not in self.dictionary.keys():
                self.dictionary[key_name] = []
            self.dictionary[key_name].append(loss_dict[key_name])
    
    def fetch_dict(self):
        return self.dictionary
    
class loss_aggregator():
    def __init__(self):
        super(loss_aggregator, self).__init__()
        self.main_loss_dict = total_loss_list()
        self.aggregated_dict = {}

    def add(self, loss_dict):
        self.main_loss_dict.update(loss_dict)

    def aggregate(self):
        for key in self.main_loss_dict.fetch_dict().keys():
            self.aggregated_dict[key] = np.mean(self.main_loss_dict.fetch_dict()[key])

        return self.aggregated_dict

In [72]:
losses = {'f1':[1,2,3,4,5,6],'f2':[10,20,30,40,50,60]}

In [74]:
loss_logger = loss_aggregator()
loss_logger.add(losses)
loss_logger.aggregate()

{'f1': 3.5, 'f2': 35.0}